## Что такое GPS?

Система глобального позиционирования, использует спутники для определения координат.

Формат данных: широта (latitude), долгота (longitude), высота (altitude).

## Зачем нужен GPS в автономных системах?

- Навигация в глобальных координатах.

- Задание маршрутов по точкам (waypoints).

- Синхронизация с картографическими данными.

## Преобразование GPS координат в декартовы

**Проблема**: GPS даёт координаты на Земном шаре, а для управления дроном нужны координаты в локальной системе.

**Решение**: Используем сферические координаты или WGS-84

In [1]:
from control import ROSInterface
from logger import get_logger
import time
import numpy as np

GPS_COORDINATES = [
    [ 8.60454360e-06, 6.21721031e-06, 1.04610503e-01],
    [ 8.67838525e-06, -6.17695277e-06,  1.04767235e-01],
    [ -3.85241178e-06, -6.26745547e-06,  1.04761751e-01],
    [ -3.94268869e-06,  6.17762800e-06,  1.04767531e-01],
]

In [2]:
# Сначала попробуем перевести в сферическую систему координат

def convert_gps(lat, lon, alt):
	R = 6378137.0 # Радиус Земли

	x = R * np.cos(lat) * np.cos(lon)
	y = R * np.cos(lat) * np.sin(lon)
	z = R * np.sin(lat)
	
	return x, y, z

In [3]:
# А теперь с помощью формулы перевода из WGS-84 (где полагается, что Земля - эллипсоид) в декартовы

def convert_gps_el(lat, lon, alt):
	a = 6378137.0 # Большая полуось
	b = 6356752.3142 # Малая полуось
	e_sq = (a ** 2 - b ** 2) / a ** 2 # Квадрат эксцентриситета
	N = a / np.sqrt(1 - e_sq * np.sin(lat) ** 2) # Радиус кривизны первого вертикала

	x = (N + alt) * np.cos(lat) * np.cos(lon)
	y = (N + alt) * np.cos(lat) * np.sin(lon)
	z = (N * (1 - e_sq) + alt)* np.sin(lat)

	return x,y,z

In [4]:
# Посмотрим для каждой из точек, что получили (исходные данные получены с некоторой погрешностью, так что сравнивать методы довольно некорректно)
for gps in GPS_COORDINATES:
  x, y, z = convert_gps_el(gps[0], gps[1], gps[2])
  print((x, y, z))
  x, y, z = convert_gps(gps[0], gps[1], gps[2])
  print((x, y, z))
  print()

(np.float64(6378137.104252701), np.float64(39.654219763664365), np.float64(54.5135648155373))
(np.float64(6378136.9996406175), np.float64(39.65421911326904), np.float64(54.88095790259598))

(np.float64(6378137.104406983), np.float64(-39.39745165500756), np.float64(54.98138411800053))
(np.float64(6378136.999638139), np.float64(-39.39745100785536), np.float64(55.351930062584444))

(np.float64(6378137.10458947), np.float64(-39.974690285092656), np.float64(-24.40672149911745))
(np.float64(6378136.999827401), np.float64(-39.974689628501054), np.float64(-24.57121011319308))

(np.float64(6378137.104596586), np.float64(39.401758365696026), np.float64(-24.978665394545896))
(np.float64(6378136.999828722), np.float64(39.40175771847914), np.float64(-25.14700861310538))



In [5]:
# Пролетим по всем точкам

drone = ROSInterface()
time.sleep(2)

drone.takeoff(6.0)
for gps in GPS_COORDINATES:
  x, y, z = convert_gps(gps[0], gps[1], gps[2])
  drone.fly_to_pose((y, z, 6.0))

drone.shutdown()

[INFO] [1765461478.112169304] [drone_controller]: ROS 2 node initialized.
[INFO] [1765461478.131526854] [drone_controller]: ROS 2 spin thread started.
[INFO] [1765461480.134049033] [drone_controller]: Takeoff to altitude 6.0
[INFO] [1765461490.052561842] [drone_controller]: Flying to pose (np.float64(39.65421911326904), np.float64(54.88095790259598), 6.0)
[INFO] [1765461522.704620148] [drone_controller]: Flying to pose (np.float64(-39.39745100785536), np.float64(55.351930062584444), 6.0)
[INFO] [1765461555.950270652] [drone_controller]: Flying to pose (np.float64(-39.974689628501054), np.float64(-24.57121011319308), 6.0)
[INFO] [1765461589.183367389] [drone_controller]: Flying to pose (np.float64(39.40175771847914), np.float64(-25.14700861310538), 6.0)
[INFO] [1765461622.412338076] [drone_controller]: Shutting down Drone Controller...
[INFO] [1765461622.412747815] [drone_controller]: Waiting for spin thread to finish...
[INFO] [1765461622.413966585] [drone_controller]: ROS 2 spin threa

## Расчёт расстояния между GPS-точками

### Формула гаверсинусов

$$
2arcsin(\sqrt{sin^{2}\frac{\phi_{2} - \phi_{1}}{2} + cos(\phi_{1}) cos(\phi_{2}) sin^{2}\frac{\lambda_{2} - \lambda_{1}}{2}}
$$

In [ ]:
def haversine(lat1, lon1, alt1, lat2, lon2, alt2):
	R = 6378137.0 # Радиус Земли
	dlat = lat2 - lat1
	dlon = lon2 - lon1
	a = np.sqrt(np.sin(dlat / 2) ** 2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon / 2) ** 2)
	return R * 2 * np.arcsin(a)

Единственная проблема - эта формула подвержена проблеме точек-антиподов, чтобы ее решить используется следующая формула:

$$
arctan{\frac {\sqrt {(cos(\phi_{2})sin(\lambda_{2} - \lambda_{1}))^{2} + (cos(\phi_{1})sin(\phi_{2}) - sin(\phi_{1})cos(\phi_{2})cos(\lambda_{2} - \lambda_{1}))^{2}}} {sin(\phi_{1})sin(\phi_{2}) + cos(\phi_{1})cos(\phi_{2})cos(\lambda_{2} - \lambda_{1})}}
$$

In [ ]:
def haversine_2(lat1, lon1, alt1, lat2, lon2, alt2):
	R = 6378137.0 # Радиус Земли
	dlon = lon2 - lon1
	a = np.sqrt((np.cos(lat2) * np.sin(dlon)) ** 2 + (np.cos(lat1) * np.sin(lat2) - np.sin(lat1) * np.cos(lat2) * np.cos(dlon)) ** 2)
	c = np.sin(lat1) * np.sin(lat2) + np.cos(lat1) * np.cos(lat2) * np.cos(dlon)
	return R * np.arctan2(a, c)

Для примера посчитаем расстояние между 2 соседними кубами (опять не забываем про погрешность в исходных данных)

In [8]:
haversine(*GPS_COORDINATES[0], *GPS_COORDINATES[1])

np.float64(79.05307308254731)

In [9]:
haversine_2(*GPS_COORDINATES[0], *GPS_COORDINATES[1])

np.float64(79.05307308254731)